In [1]:

file_id = '1rDltFC9NikOhuEWKWCpVSeQ6pW3LRJed'

In [2]:
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
download = drive.CreateFile({'id': file_id})
# Download the file to a local disc
download.GetContentFile('data.csv')

In [4]:
import pandas as pd
import numpy as np

In [5]:
# Read the CSV file without headers
df = pd.read_csv('data.csv',sep = '|')

# Assign headers 'class' and 'sentence'
df.columns = [ 'sentence','class']

# Now you can use the modified DataFrame 'df' for further processing


In [6]:
df

,sentence,class
0,My bank account number is not correct. Please ...,Account number is not Correct
1,The installment for this month has not been re...,Installment not received
2,I have not received the money for the previous...,Installment not received
3,The account number mentioned on the portal is ...,Account number is not Correct
4,I haven't received the installment for the cur...,Installment not received
...,...,...
160,My installment has not arrived and I need to p...,Installment not received
161,"I haven't received the installment, which is m...",Installment not received
162,I need to procure mushroom spawn for my mushro...,Installment not received
163,"The installment hasn't arrived, and I need to ...",Installment not received


In [7]:
!pip install transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00


In [8]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

In [9]:
# Specify the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df['class'].nunique())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [10]:
df_full = df.copy()

In [11]:
df= df_full.sample(df_full.shape[0])
df['class'].nunique()
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['class'])

In [12]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [13]:
max_length = 128
train_encodings = tokenizer(list(train_df['sentence']), truncation=True, padding=True, max_length=max_length)

In [14]:
val_encodings = tokenizer(list(val_df['sentence']), truncation=True, padding=True, max_length=max_length)


In [15]:
# Convert the tokenized data to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(train_df['label'].values))

In [16]:
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            torch.tensor(val_df['label'].values))

In [17]:
# Create data loaders for training and validation
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)


In [18]:
# Set the device for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)
from sklearn.metrics import accuracy_score, f1_score

# Training loop
epochs = 15
model.to(device)
model.train()

prev_loss = float('inf')
no_decrease_count = 0

for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_predicted_labels = []
    all_true_labels = []

    for batch in train_dataloader:
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        predicted_labels = torch.argmax(outputs.logits, dim=1)
        true_labels = inputs['labels']
        total_loss += loss.item()
        total_correct += (predicted_labels == true_labels).sum().item()
        total_samples += len(true_labels)
        all_predicted_labels.extend(predicted_labels.tolist())
        all_true_labels.extend(true_labels.tolist())

    # Calculate accuracy and F1 score
    epoch_loss = total_loss / len(train_dataloader)
    epoch_accuracy = total_correct / total_samples
    epoch_f1_score = f1_score(all_true_labels, all_predicted_labels, average='weighted')

    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.4f} - F1 Score: {epoch_f1_score:.4f}")

    # Check if the loss decreased
    if epoch_loss >= prev_loss:
        no_decrease_count += 1
    else:
        no_decrease_count = 0

    prev_loss = epoch_loss

    # Stop training if there is no decrease in loss for 2 consecutive iterations
    if no_decrease_count >= 2:
        print("No decrease in loss for 2 consecutive iterations. Stopping training.")
        break


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/15 - Loss: 1.0574 - Accuracy: 0.4242 - F1 Score: 0.4202
Epoch 2/15 - Loss: 0.7780 - Accuracy: 0.7500 - F1 Score: 0.6952
Epoch 3/15 - Loss: 0.5212 - Accuracy: 0.8939 - F1 Score: 0.8825
Epoch 4/15 - Loss: 0.3214 - Accuracy: 0.9773 - F1 Score: 0.9767
Epoch 5/15 - Loss: 0.1612 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 6/15 - Loss: 0.0883 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 7/15 - Loss: 0.0592 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 8/15 - Loss: 0.0418 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 9/15 - Loss: 0.0321 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 10/15 - Loss: 0.0258 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 11/15 - Loss: 0.0213 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 12/15 - Loss: 0.0192 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 13/15 - Loss: 0.0160 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 14/15 - Loss: 0.0142 - Accuracy: 1.0000 - F1 Score: 1.0000
Epoch 15/15 - Loss: 0.0121 - Accuracy: 1.0000 - F1 Score: 1.0000


In [19]:

# Evaluation
model.eval()
all_predicted_labels = []
all_true_labels = []

with torch.no_grad():
    for batch in val_dataloader:
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=1)
        all_predicted_labels.extend(predicted_labels.cpu().numpy())
        all_true_labels.extend(inputs['labels'].cpu().numpy())

# Calculate accuracy and F1 score
val_accuracy = accuracy_score(all_true_labels, all_predicted_labels)
val_f1_score = f1_score(all_true_labels, all_predicted_labels, average='weighted')

print(f"Validation Accuracy: {val_accuracy:.4f}")
print(f"Validation F1 Score: {val_f1_score:.4f}")


Validation Accuracy: 1.0000
Validation F1 Score: 1.0000


In [21]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [22]:
!git config --global credential.helper store

In [23]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [31]:
!git clone https://huggingface.co/GautamR/model_grievance_class

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.


In [35]:
# Commit as usual
%cd model_grievance_class
!git lfs install
!git config --global user.email "gautam@samagragovernance.in"
# Tip: using the same email than for your huggingface.co account will link your commits to your profile
!git config --global user.name "GautamR"
!echo "hello" >> README.md
!git add . && git commit -m "Update from $USER"

Updated git hooks.
Git LFS initialized.
[main effb583] Update from
 1 file changed, 2 insertions(+)
 create mode 100644 README.md


In [36]:
model.save_pretrained('.')
tokenizer.save_pretrained('.')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.txt',
 './added_tokens.json')

In [37]:
!chmod +x .git/hooks/pre-push

In [38]:
!git add .
!git commit -m "Initial commit"
!git push

[main 449a958] Initial commit
 5 files changed, 30584 insertions(+)
 create mode 100644 config.json
 create mode 100644 pytorch_model.bin
 create mode 100644 special_tokens_map.json
 create mode 100644 tokenizer_config.json
 create mode 100644 vocab.txt
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 4 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (10/10), 109.55 KiB | 5.22 MiB/s, done.
Total 10 (delta 1), reused 0 (delta 0)
remote: -------------------------------------------------------------------------
remote: Your push was accepted, but with warnings: 
remote: - Warning: empty or missing yaml metadata in repo card
remote: help: https://huggingface.co/docs/hub/model-cards#model-card-metadata
remote: -------------------------------------------------------------------------
remote: -------------------------------------------------------------------------
remote: Please find the documentation at:
remote: https://

## GPT data creation

In [11]:
prompt =  ''' Pradhan Mantri Kisan Samman Nidhi (PIV-KISAN) is a new Central Sector Scheme to
provide income support to all landholding farmers' families in the country to
supplement their financial needs for procuring various inputs related to agriculture and
allied activities as well as domestic needs. Under the Scheme, the entire financial
liability towards transfer of benefit to targeted beneficiaries will be borne by
Government of India. We are building a classifier for training a grievance redressal system for it which classifies feedback/grievance provided to it into 3 buckets.
The  buckets are :
['Account number is not Correct' ,'Installment not received', 'Others']

'Account number not correct' is for those grievances/ feedback which talk about the bank account not being correct on the portal for these farmers.
'Installment not received' is for those feedback which talk about the farmer not receiving the installment/money for that month.
'Others'  cover all other types of grievances - including 'Gender being wrong ,'Online Application pnding approval,' pther Payment Related issues','Problem in Adhaar Correction' ,'problem in bio-metric based e-kyc','Problem in OTP based e-kyc', 'Tansaction Failed' etc etc.

Can you create some training data in an Indian context ( in the form of a pipe separated file ) on which I can train BERT  to be able to classify the grievances. I need two columns - 'grievance' (the text from the user with the feedback) and 'class' (the classification into the above 3 buckets)  - can you provide 30 examples for each bucket in a csv format. Can you also include some bigger grievences of 4-5 sentences.  while most are around 2 sentences'''